In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import joblib
import imageio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from skimage.transform import resize

In [ ]:
from params import *

In [ ]:
IMG_PATH = "../../../data/nfl/"

In [ ]:
df = pd.read_csv(DATA_PATH + 'df_train.csv')

In [ ]:
def adapt_to_shape(box, shape):
    if box[0] < 0:
        box[1] -= box[0]
        box[0] = 0
    elif box[2] < 0:
        box[3] -= box[2]
        box[2] = 0
    elif box[1] >= shape[0]:
#         print(box, shape)
        diff = box[1] - shape[0]
        box[1] -= diff
        box[0] -= diff
#         print(box)
    elif box[3] >= shape[1]:
#         print(box, shape)
        diff = box[3] - shape[1]
        box[3] -= diff
        box[2] -= diff
#         print(box)
    else:
        print("BUG")
    
    return box

In [ ]:
def crop_helmets(image_name, df, size=64, root="", out_dir=""):
    df_img = df[df['image_name'] == image_name].copy()

    image = cv2.imread(root + image_name)
    
    boxes = df_img[['x', 'w', 'y', 'h']].values
    boxes[:, 1] += boxes[:, 0]
    boxes[:, 3] += boxes[:, 2]
    
    crop_names = []
    for i in range(len(df_img)):
        box = boxes[i]
        box = extend_box(box, size=size)
        
        img_crop = image[box[2] : box[3], box[0] : box[1]]
    
        name = image_name[:-4] + f'_{i:02d}.png'
            
        try:
            if img_crop.shape == (64, 64, 3):
                cv2.imwrite(out_dir + name, img_crop)
            else:
                name = ""
        except:
            name = ""
        
        crop_names.append(name)
    
    return crop_names

In [ ]:
crop_names = []
for img in tqdm(df['image_name'].unique()):
    names = crop_helmets(
        img,
        df,
        root=IMG_PATH,
        out_dir=CROP_PATH,
    )
    
    crop_names += names

In [ ]:
df['crop_name'] = crop_names

In [ ]:
df.to_csv(DATA_PATH + 'df_train.csv', index=False)